In [5]:
import pandas as pd, numpy as np, os, requests, warnings
import config
from unidecode import unidecode
from bs4 import BeautifulSoup, Comment
warnings.filterwarnings("ignore")

pd.set_option("display.max_columns", None)

In [6]:
sav25 = pd.read_csv("data/sav25.csv")
sav25 = sav25.loc[sav25["game_type"] == "R"]

In [7]:

sav25["description"].value_counts()

description
ball                       238212
foul                       127371
hit_into_play              124899
called_strike              115574
swinging_strike             74119
blocked_ball                15139
foul_tip                     7303
swinging_strike_blocked      3997
automatic_ball               2348
hit_by_pitch                 1928
foul_bunt                    1247
missed_bunt                   208
automatic_strike               96
pitchout                       54
bunt_foul_tip                  33
Name: count, dtype: int64

In [ ]:
def hitter_addOns(sav):
  sav["dk_points"] = sav["events"].map(config.points_dict).fillna(0)
  sav["is_at_bat"] = sav["events"].isin(config.at_bat_list)
  sav["is_pa"] = sav["events"].isin(config.plate_appearance_list)
  
  sav["is_swinging_strike"] = sav["description"].isin(config.swinging_strike_list)
  sav["is_whiff"] = sav["description"].isin(config.whiff_list)
  sav["is_swing"] = sav["description"].isin(config.swing_list)
  sav["is_called_strike"] = sav["description"].isin(config.called_strike_list)

  sav["in_zone"] = sav["zone"].between(1, 9)
  sav["out_of_zone"] = ~sav["in_zone"]

  sav["is_chase"] = sav["is_swing"] & sav["out_of_zone"]

  return sav

In [14]:
df = hitter_addOns(sav25)
df.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,fielder_2,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length,estimated_slg_using_speedangle,delta_pitcher_run_exp,hyper_speed,home_score_diff,bat_score_diff,home_win_exp,bat_win_exp,age_pit_legacy,age_bat_legacy,age_pit,age_bat,n_thruorder_pitcher,n_priorpa_thisgame_player_at_bat,pitcher_days_since_prev_game,batter_days_since_prev_game,pitcher_days_until_next_game,batter_days_until_next_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,arm_angle,attack_angle,attack_direction,swing_path_tilt,intercept_ball_minus_batter_pos_x_inches,intercept_ball_minus_batter_pos_y_inches,batter_name,pitcher_name,dk_points,is_at_bat,is_pa,is_swinging_strike,is_whiff
14245,FF,2025-09-28,95.7,-2.15,5.21,"Weissert, Greg",678009,669711,field_out,hit_into_play,NaN,NaN,NaN,NaN,1.0,Parker Meadows flies out to left fielder Jarre...,R,L,R,BOS,DET,X,7.0,fly_ball,0,0,2025,-0.71,0.93,-0.40,3.08,NaN,700242.0,628451.0,2,9,Top,96.06,86.56,NaN,NaN,NaN,NaN,6.169368,-139.262184,-2.088523,-10.601017,31.025947,-19.832498,3.45,1.69,290.0,83.8,37.0,94.9,2370.0,5.8,776151,657136,663993,666152,681987,686765,680776,678882,677800,54.67,0.009,0.008,0.0,1.0,0.0,0.0,3.0,74,1,4-Seam Fastball,4,3,3,4,3,4,3,4,Infield shade,Strategic,226.0,0.146,-0.282,69.0,7.1,0.013,0.282,88.0,1,-1,0.854296,0.145704,30,25,30,26,1,4,2.0,1.0,4.0,2.0,1.56,0.71,-0.71,20.9,5.991833,-1.319512,28.782516,41.559201,30.599805,Parker Meadows,Greg Weissert,0.0,True,True,False,False
14246,FF,2025-09-28,95.1,-1.91,5.10,"Weissert, Greg",668670,669711,strikeout,called_strike,NaN,NaN,NaN,NaN,6.0,Jake Rogers called out on strikes.,R,R,R,BOS,DET,S,2.0,NaN,0,2,2025,-0.93,0.93,0.57,2.59,NaN,700242.0,628451.0,1,9,Top,NaN,NaN,NaN,NaN,NaN,NaN,8.617368,-138.217784,-3.023825,-13.845463,30.454633,-19.764496,3.37,1.53,NaN,NaN,NaN,95.6,2309.0,6.6,776151,657136,663993,666152,681987,686765,680776,678882,677800,53.86,NaN,0.000,0.0,1.0,0.0,0.0,NaN,73,4,4-Seam Fastball,4,3,3,4,3,4,3,4,Standard,Standard,229.0,0.149,-0.160,NaN,NaN,NaN,0.160,NaN,1,-1,0.711180,0.288820,30,30,30,30,1,3,2.0,9.0,4.0,9.0,1.59,0.93,0.93,20.5,NaN,NaN,NaN,NaN,NaN,Jake Rogers,Greg Weissert,0.0,True,True,False,False
14247,FF,2025-09-28,95.4,-1.99,5.22,"Weissert, Greg",668670,669711,NaN,foul,NaN,NaN,NaN,NaN,2.0,Foul,R,R,R,BOS,DET,S,NaN,NaN,0,2,2025,-0.85,1.14,0.06,3.30,NaN,700242.0,628451.0,1,9,Top,NaN,NaN,NaN,NaN,NaN,NaN,7.213709,-138.740928,-1.962713,-12.545880,30.045431,-17.188392,3.47,1.60,246.0,80.3,45.0,94.6,2314.0,5.8,776151,657136,663993,666152,681987,686765,680776,678882,677800,54.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73,3,4-Seam Fastball,4,3,3,4,3,4,3,4,Standard,Standard,211.0,0.014,0.000,63.5,6.2,NaN,0.000,88.0,1,-1,0.711180,0.288820,30,30,30,30,1,3,2.0,9.0,4.0,9.0,1.36,0.85,0.85,22.9,2.871131,31.805044,22.266527,37.478847,15.582717,Jake Rogers,Greg Weissert,0.0,False,False,False,False
14248,SL,2025-09-28,84.8,-2.33,4.72,"Weissert, Greg",668670,669711,NaN,swinging_strike,NaN,NaN,NaN,NaN,5.0,Swinging Strike,R,R,R,BOS,DET,S,NaN,NaN,0,1,2025,0.32,0.62,0.14,2.59,NaN,700242.0,628451

In [ ]:
df_stats = (
  df.groupby(["batter_name", "batter"]).agg(
    dk_points_total = ("dk_points", "sum"),
    at_bats = ("is_at_bat", "sum")
  )
  .reset_index()
  .sort_values(by="dk_points_total", ascending=False)
)

df_stats["dk_pts_avg"] = df_stats["dk_points_total"] / df_stats["at_bats"]

In [ ]:
df_stats = df_stats.loc[df_stats["at_bats"] > 160]

In [ ]:
df_stats.sort_values(by="dk_pts_avg", ascending=False).head(16)

In [ ]:
df.loc[df["pitcher"] == 676664].groupby("pitch_type")["release_speed"].mean()

In [ ]:
l_91_93 = df.loc[
  (df["release_speed"] < 94)
  & (df["release_speed"] > 90)
  & (df["pitch_type"] == "FF")
  & (df["p_throws"] == "L")
]

In [ ]:
df_l = (
  l_91_93.groupby(["batter_name", "batter"]).agg(
    dk_points_total = ("dk_points", "sum"),
    at_bats = ("is_at_bat", "sum")
  )
  .reset_index()
  .sort_values(by="dk_points_total", ascending=False)
)

df_l["dk_pts_avg"] = df_l["dk_points_total"] / df_l["at_bats"]

In [ ]:
df_l.loc[df_l["at_bats"] > 10].sort_values(by="dk_pts_avg", ascending=False)